- Assignments:

1. Weather model:

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:
- First, load the dataset from the weatherinszeged table from Thinkful's database.

In [1]:
# Import some libraries and packages:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import acf
from scipy.stats import bartlett
from scipy.stats import levene
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Connect to dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged', con = engine)
engine.dispose()

In [3]:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.47,7.39,0.89,14.12,251.00,15.83,0.00,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.36,7.23,0.86,14.26,259.00,15.83,0.00,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.38,9.38,0.89,3.93,204.00,14.96,0.00,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.29,5.94,0.83,14.10,269.00,15.83,0.00,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.76,6.98,0.83,11.04,259.00,15.83,0.00,1016.51,Partly cloudy throughout the day.


- Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. 
- What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

In [4]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed:
X = df[['humidity', 'windspeed']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        17:26:32   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- R-squared is a proportion (between 0 and 1) that expresses how much variance in the outcome variable is explained by the explanatory variables in the model. 
- Adjusted R-squared does the same job as R-squared, but it is adjusted according to the number of features included in the model. Because,R-squared is biased as it tends to increase with the number of explanatory variables.
- So, in this model: R-squared and adjusted R-squared of the model both are 0.288.
- I think it is not satisfactory, because both R-squared and adjusted R-squared have low values around 0.29 and the higher values are better. So, 71% of the variation in the target variable is unexplained by the features or explanatory variables in the model. So, the results are not satisfactory. 
- however in some cases, it is hard to explain target variables also, the best models can explain a very low portion of the information in the target. So, in these cases, satisfaction of the results is a relative success.

- Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [5]:
# Y is the target variable
Y = (df['apparenttemperature']-df['temperature'])

# This is the interaction between humidity and windspeed
df["humidity_windspeed"] = df.humidity * df.windspeed

# X is the feature set
X = df[['humidity', 'windspeed', 'humidity_windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        17:26:32   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- So, in this model: R-squared and adjusted R-squared of the model both are 0.341.
- By Including the interaction the R-squared and adjusted R-squared values are increased. So, 64% of the variation in the target variable is unexplained by the explanatory variables in the model which is decreased compare to previous one. So, this is an improvement upon the previous model.

- Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [6]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed, and visibility:
X = df[['humidity', 'windspeed', 'visibility']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        17:26:33   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- In this model, R-squared is 0.304 and Adjust R-squared is 0.303. Comparing to first model, Both R-squared and adjusted R-squared increased by adding visibility as an explanatory variable in the model and improved the model.
- Comparing this model with interaction term model, this model's improvement is lower. Interaction term can explain more portion of target variable, so it is more useful than visibility to explain the information in the target.

- Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.?

- For both AIC and BIC, the lower the value the better.

In the first model; both AIC and BIC are 340900, In the second model; both AIC and BIC are 333400, and for the third model, both AIC and BIC are 338800. So, the second model with lowest  AIC and BIC is the best of these three models.

2. House prices model:

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- Load the houseprices data from Thinkful's database.

In [7]:
# Connect to the data base:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

In [8]:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.00,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


- Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC?

In [9]:
# Convert categorical columns to numeric by using dummies:
categorical = df.select_dtypes(include=['object'])
dummies = pd.get_dummies(categorical, drop_first=True)
df = df.drop(categorical, axis=1)
df = pd.concat([df,dummies], axis=1)

In [10]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set:
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'exterqual_TA', 'kitchenqual_TA']]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     686.7
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        17:26:39   Log-Likelihood:                -17477.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1452   BIC:                         3.501e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -6.278e+04   7313.170     -8.584      0.000   -7.71e+04   -4.84e+04
overallqual     2.001e+04   1199.489     16.680      0.000    1.77e+04    2.24e+04
grlivarea         45.4666      2.457     18.505      0.000      40.647      50.286
garagecars        1.3e+04   2992.143      4.344      0.000    7129.524    1.89e+04
garagearea        16.9487     10.323      1.642      0.101      -3.301      37.199
totalbsmtsf       31.0423      2.868     10.825      0.000      25.417      36.667
exterqual_TA   -9898.1910   3146.166     -3.146      0.002   -1.61e+04   -3726.675
kitchenqual_TA -9741.9446   2776.148     -3.509      0.000   -1.52e+04   -4296.256
==============================================================================
Omnibus:                      429.417   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            43903.389
Skew:                          -0.225   Prob(JB):                         0.00
Kurtosis:                      29.861   Cond. No.                     1.51e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In this model:

- F_test is 686.7 and p_value is 0
- R-square and Adjust R-squared both are around 0.77
- AIC is 34970 and BIC is 35010

According to the R-squared and Adjust R-squared almost 23% of the variance in the target variable is unexplained by explanatory variable in the model. So, there is a room for improvement.

- Do you think your model is satisfactory? If so, why?

I think it is not satisfactory, because both R-squared and adjusted R-squared have space to improve and the higher values are better. So, 23% of the variation in the target variable is unexplained by the features or explanatory variables in the model. So, the results are not satisfactory enough. 

- In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables?

- In order to improve the goodness of fit of your model, I add totalsf and yearbuilt variables and remove garagearea and exterqual_TA.

In [19]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set:
df['totalsf'] = df['totalbsmtsf'] + df['firstflrsf'] + df['secondflrsf']

X = df[['overallqual', 'grlivarea', 'garagecars', 'totalsf', 'yearbuilt', 'kitchenqual_TA']]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     818.5
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        17:34:59   Log-Likelihood:                -17466.
No. Observations:                1460   AIC:                         3.495e+04
Df Residuals:                    1453   BIC:                         3.498e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -5.569e+05   8.71e+04     -6.391      0.000   -7.28e+05   -3.86e+05
overallqual     1.887e+04   1194.167     15.805      0.000    1.65e+04    2.12e+04
grlivarea         20.1800      4.088      4.937      0.000      12.161      28.199
garagecars      1.411e+04   1811.505      7.787      0.000    1.06e+04    1.77e+04
totalsf           30.3603      2.767     10.973      0.000      24.933      35.788
yearbuilt        251.0505     45.291      5.543      0.000     162.208     339.893
kitchenqual_TA -1.121e+04   2455.069     -4.567      0.000    -1.6e+04   -6397.532
==============================================================================
Omnibus:                      437.638   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            47760.493
Skew:                          -0.242   Prob(JB):                         0.00
Kurtosis:                      31.016   Cond. No.                     3.21e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.21e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In this model:

- F_test is 818.5 and p_value is 0 and compare to the first model (F_test is 686.7) is improved. Because the model with higher F statistic is superior to the other one.
- R-square and Adjust R-squared both are around 0.77. So, almost 33% of the variance in the target variable is unexplained by explanatory variable in the model. So, compare to the first model, this model is not improved.
- AIC is 34950 and BIC is 34980. Comparing with first model, AIC and BIC both are less then first model. So,this model is better than the first model.

- So according to above, this model has a better goodness of fit than the first model. 